In [2]:
%load_ext watermark
import pandas as pd
import numpy as np

from review_methods_tests import collect_vitals, find_missing, find_missing_loc_dates
from review_methods_tests import use_gfrags_gfoams_gcaps, make_a_summary,combine_survey_files

# Testing data models

The methods used in the version of the federal report were tested, but their was not a specific set of validation criteria beforehand. Test were done as the work progressed. This wasted alot of time

here we test the land use and survey data models.

1. is the land use data complete for each survey location?
2. does the survey data aggregate correctly to sample level?
   * what happens to objects with a quantity of zero?
   * aggregating to cantonal, municipal or survey area
     * are all locations included?
     * are lakes and rivers distinguished?
3. Does the aggregated data for iqaasl match the federal report?

### Gfoams, Gfrags, Gcaps

These are aggregate groups. It is difficult to infer how well a participant differentiates between size or use of the following codes.

1. Gfrags: G79, G78, G75
2. Gfoams: G81, G82, G76
3. Gcaps: G21, G22, G23, G24

These aggregate groups are used when comparing values between sampling campaigns.

### Sampling campaigns

The dates of the sampling campaigns are expanded to include the surveys that happened between large organized campaigns. The start and end dates are defined below.

__Attention!!__ The codes used for each survey campaign are different. Different groups organized and conducted surveys using the MLW protocol. The data was then sent to us.

__MCBP:__ November 2015 - November 2016. The initial sampling campaign. Fragmented plastics (Gfrags/G79/G78/G76) were not sorted by size. All unidentified hard plastic items were classified in this manner.

* start_date = 2015-11-15
* end_date = 2017-03-31

__SLR:__ April 2017 - May 2018. Sampling campaign by the WWF. Objects less than 2.5 cm were not counted.

* start_date = 2017-04-01
* end_date = 2020-03-31

__IQAASL:__ April 2020 - May 2021. Sampling campaign mandated by the Swiss confederation. Additional codes were added for regional objects.

* start_date = 2020-04-01
* end_date = 2021-05-31

__Plastock (not added yet):__ January 2022 - December 2022. Sampling campaign from the Association pour la Sauvegarde du Léman. Not all objects were counted, They only identified a limited number of objects.

### Feature type

The feature type is a label that applies to general conditions of use for the location and other locations in the region

* r: rivers: surveys on river banks
* l: lake: surveys on the lake shore
* p: parcs: surveys in recreational areas

### Parent boundary

Designates the larger geographic region of the survey location. For lakes and rivers it is the name of the catchment area or river basin. For parcs it is the the type of park ie.. les Alpes. Recall that each feature has a name, for example Alpes Lépontines is the the name of a feature in the geographic region of _Les Alpes_.

In [3]:
# period dates
period_dates = {
    "mcbp":["2015-11-15", "2017-03-31"],
    "slr": ["2017-04-01", "2020-02-28"],
    "iqaasl": ["2020-03-01", "2021-05-31"],
    "2022": ["2021-06-01", "2022-12-01"]
}
code_cols = ['material', 'description', 'source', 'parent_code', 'single_use', 'groupname']

group_by_columns = [
    'loc_date', 
    'date', 
    'feature_name', 
    'slug',     
    'parent_boundary',
    'length',
    'groupname',
    'city',
    'code', 
]
agg_this = {
    "quantity":"sum",
    "pcs_m": "sum"
}

survey_data = [
    "data/end_process/after_may_2021.csv",
    "data/end_process/iqaasl.csv",
    "data/end_process/mcbp.csv",
    "data/end_process/slr.csv",
]

source_data = "data/end_process/new_allx.csv"

code_data =  "data/end_process/codes.csv"
beach_data = "data/end_process/beaches.csv"
land_cover_data = "data/end_process/land_cover.csv"
land_use_data = "data/end_process/land_use.csv"
street_data = "data/end_process/streets.csv"
intersection_attributes = "data/end_process/river_intersect_lakes.csv"
surveys = combine_survey_files(survey_data)
codes = pd.read_csv(code_data).set_index("code")
beaches = pd.read_csv(beach_data).set_index("slug")
land_cover = pd.read_csv(land_cover_data)
land_use = pd.read_csv(land_use_data)
streets = pd.read_csv(street_data)
river_intersect_lakes = pd.read_csv(intersection_attributes)

## Aggregate a set of data by sample (location and date)

Use the loc_date column in the survey data. Use the IQAASL period and test against the federal report

### Before aggregating does the number of locations, cities, samples and quantity match the federal report?

__The feature types include lakes and rivers, alpes were condsidered separately__

From https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/lakes_rivers.html#

1. cities = yes
2. samples = yes
3. locations = yes
4. quantity = No it is short 50 pieces

In [4]:
period = "iqaasl"
survey_areas = ["rhone", "ticino", "linth", "aare"]
start, end = [*period_dates[period]]
survey_data = surveys[surveys.parent_boundary.isin(survey_areas)].copy()

def slice_data_by_date(data, start, end):
    mask = (data.date >= start) & (data.date <= end)
    return data[mask]

feature_d= slice_data_by_date(survey_data.copy(), start, end)


feature_data = use_gfrags_gfoams_gcaps(feature_d.copy(), codes)
feature_vitals = collect_vitals(feature_d)
print(make_a_summary(feature_vitals))


    Number of objects: 54694
    
    Median pieces/meter: 0.0
    
    Number of samples: 386
    
    Number of unique codes: 235
    
    Number of sample locations: 143
    
    Number of features: 28
    
    Number of cities: 77
    
    Start date: 2020-03-08
    
    End date: 2021-05-12
    
    


### aggregate to sample

The assessments are made on a per sample basis. That means that we can look at an individual object value at each sample. The sum of all the individual objects in a survey is the total for that survey. Dividing the totals by the length of the survey gives the assessment metric: _pieces of trash per meter_.

1. Are the quantiles of the current data  = to the federal report? Yes
2. Are the material totals = to the federal report? No,plastics if off by 50 pcs

In [5]:
import pandas as pd
from typing import Type, Optional
from typing import List, Dict, Union

def aggregate_dataframe(
    df: pd.DataFrame, 
    groupby_columns: List[str], 
    aggregation_functions: Dict[str, Union[str, callable]],
    index: bool = False
) -> pd.DataFrame:
    """
    Aggregate specified columns in a Pandas DataFrame using given aggregation functions.

    Args:
        df (pd.DataFrame): The input DataFrame.
        groupby_columns (List[str]): List of column names to group by.
        aggregation_functions (Dict[str, Union[str, callable]]): 
            A dictionary where keys are column names to aggregate, 
            and values are either aggregation functions (e.g., 'sum', 'mean', 'max', 'min')
            or custom aggregation functions (callable functions).
        index (bool, optional): Whether to use the groupby columns as an index.
            Default is False.

    Returns:
        pd.DataFrame: A new DataFrame with aggregated values.
    """
    grouped = df.groupby(groupby_columns, as_index=index).agg(aggregation_functions)
    
    return grouped
    
def merge_dataframes_on_column_and_index(
    left_df: pd.DataFrame, 
    right_df: pd.DataFrame, 
    left_column: str, 
    how: str = 'inner',
    validate: str = 'many_to_one'
) -> pd.DataFrame:
    """
    Merge two DataFrames where the left DataFrame is merged on a specified column and 
    the right DataFrame is merged on its index.

    Args:
        left_df (pd.DataFrame): The left DataFrame to be merged.
        right_df (pd.DataFrame): The right DataFrame to be merged on its index.
        left_column (str): The column in the left DataFrame to merge on.
        how (str, optional): The type of merge to be performed ('left', 'right', 'outer', or 'inner'). 
            Default is 'inner'.
        validate (str, optional): Whether to perform merge validation checks. 
            Default is 'many_to_one'.

    Returns:
        pd.DataFrame: A new DataFrame resulting from the merge operation.
    """
  
    merged_df = left_df.merge(right_df, left_on=left_column, right_index=True, how=how)
    return merged_df

code_result_df = aggregate_dataframe(feature_data.copy(), group_by_columns, agg_this)
sample_totals = aggregate_dataframe(code_result_df.copy(), ["loc_date", "slug", "parent_boundary"], agg_this)
sample_totals.pcs_m.describe()

count    386.000000
mean       3.952073
std        7.063422
min        0.020000
25%        0.822500
50%        1.895000
75%        3.865000
max       66.170000
Name: pcs_m, dtype: float64

In [5]:
merged_result = merge_dataframes_on_column_and_index(code_result_df.copy(), codes["material"], 'code', how='inner', validate=True)
materials = aggregate_dataframe(merged_result.copy(), ["material"], {"quantity":"sum"})
materials

,material,quantity
0,chemicals,140
1,cloth,343
2,glass,2919
3,metal,1874
4,paper,1527
5,plastic,47093
6,rubber,390
7,unidentified,2
8,wood,406


In [24]:
def get_top_x_records_with_max_quantity(df, quantity_column, id_column, x):
    """
    Get the top x records with the greatest quantity and their associated ID from a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        quantity_column (str): The name of the quantity column.
        id_column (str): The name of the ID column.
        x (int): The number of records to return.

    Returns:
        pd.DataFrame: A DataFrame with the top x records having the greatest quantity and the associated ID.
    """
    # Sort the DataFrame by the quantity column in descending order, take the top x records, and select the ID column
    top_x_records = df.nlargest(x, quantity_column)[[id_column, quantity_column]]
    return top_x_records[id_column].values



code_totals = aggregate_dataframe(code_result_df.copy(), ["code"], {"quantity":"sum", "pcs_m":"median"})
abundant = get_top_x_records_with_max_quantity(code_totals.copy(), "quantity", "code", 10)
print(abundant)

['G27' 'Gfrags' 'Gfoams' 'G30' 'G67' 'G200' 'G112' 'Gcaps' 'G74' 'G95']


In [35]:
def count_objects_with_positive_quantity(df):
    """
    Count how many times each object had a quantity greater than zero in a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with columns 'sample,' 'object,' and 'quantity.'

    Returns:
        pd.Series: A Series with the count of positive quantity occurrences for each object.
    """
    # Filter the DataFrame to include rows where quantity is greater than zero
    positive_quantity_df = df[df['quantity'] > 0]

    # Count the occurrences of positive quantities for each object
    object_counts = positive_quantity_df['code'].value_counts()

    return object_counts
code_r = aggregate_dataframe(code_result_df, ["loc_date", "code"], {"quantity":"sum"})

result = count_objects_with_positive_quantity(code_r)/386
common = result[result >= .5]

most_common = list(set([*common.index, *abundant]))

the_most_common = code_totals[code_totals.code.isin(most_common)].copy()

import pandas as pd
from typing import List, Dict

def calculate_object_occurrence_rates(
    df: pd.DataFrame,
    objects_to_check: List[str],
    y: int,
    j: int
) -> Dict[str, float]:
    """
    Calculate the rate of occurrence for each object in a group of objects 'X' for a given quantity 'y' and number of samples 'j'
    from a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with columns 'sample,' 'object,' and 'quantity.'
        objects_to_check (List[str]): The list of objects to calculate occurrence rates for.
        y (int): The minimum quantity required for objects to be considered.
        j (int): The total number of samples.

    Returns:
        Dict[str, float]: A dictionary where keys are objects and values are the rates of occurrence for each object.
    """
    # Filter the DataFrame to include rows where 'object' is in 'objects_to_check' and quantity is greater than or equal to 'y'
    filtered_df = df[(df['code'].isin(objects_to_check)) & (df['quantity'] >= y)]

    # Calculate the occurrence rates for each object
    occurrence_rates = {}
    for obj in objects_to_check:
        object_filtered_df = filtered_df[filtered_df['code'] == obj]
        rate = len(object_filtered_df) / j if j > 0 else 0
        occurrence_rates[obj] = rate

    return occurrence_rates


occurrence_rates = calculate_object_occurrence_rates(code_result_df, most_common, 1, 386)
occurrence_rates

{'G74': 0.694300518134715,
 'G200': 0.6502590673575129,
 'G67': 0.6968911917098446,
 'G112': 0.3082901554404145,
 'G30': 0.8523316062176166,
 'Gfoams': 0.9119170984455959,
 'Gfrags': 1.2642487046632125,
 'G95': 0.5077720207253886,
 'Gcaps': 1.2202072538860103,
 'G89': 0.5207253886010362,
 'G27': 0.8782383419689119,
 'G178': 0.5233160621761658}

In [38]:
code_result_df[code_result_df.code == "Gfrags"]

,loc_date,date,feature_name,slug,parent_boundary,length,groupname,city,code,quantity,pcs_m
3,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,linth,30,plastic pieces,Schmerikon,Gfrags,6,0.20
149,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,linth,32,plastic pieces,Schmerikon,Gfrags,0,0.00
226,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,linth,33,plastic pieces,Schmerikon,Gfrags,1,0.03
237,"('aare-limmatspitz', '2020-07-13')",2020-07-13,aare,aare-limmatspitz,aare,100,plastic pieces,Gebenstorf,Gfrags,2,0.02
385,"('aare-limmatspitz', '2020-07-13')",2020-07-13,aare,aare-limmatspitz,aare,114,plastic pieces,Gebenstorf,Gfrags,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
88239,"('zurichsee_wollishofen_langendorfm', '2020-11...",2020-11-12,zurichsee,zurichsee_wollishofen_langendorfm,linth,100,plastic pieces,Zürich,Gfrags,2,0.02
88248,"('zurichsee_wollishofen_langendorfm', '2020-12...",2020-12-10,zurichsee,zurichsee_wollishofen_langendorfm,linth,66,plastic pieces,Zürich,Gfrags,2,0.03
88387,"('zurichsee_wollishofen_langendorfm', '2020-12...",2020-12-10,zurichsee,zurichsee_wollishofen_langendorfm,linth,68,plastic pieces,Zürich,Gfrags,0,0.00
88626,"('zurichsee_wollishofen_langendorfm', '2021-01...",2021-01-10,zurichsee,zurichsee_wollishofen_langendorfm,linth,43,plastic pieces,Zürich,Gfrags,0,0.00


In [39]:
group_by_columns

['loc_date',
 'date',
 'feature_name',
 'slug',
 'parent_boundary',
 'length',
 'groupname',
 'city',
 'code']

In [8]:
%watermark -a hammerdirt-analyst -co --iversions

Author: hammerdirt-analyst

conda environment: cantonal_report

numpy : 1.25.2
pandas: 2.0.3

